In [1]:
import os

In [2]:
%pwd

'f:\\Machine_Learning\\Red_wine_ML\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'f:\\Machine_Learning\\Red_wine_ML'

In [5]:
from dataclasses import dataclass
from pathlib import  Path

@dataclass(frozen= True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path 

In [6]:
from Red_Wine.constants import *
from Red_Wine.utils.common import  read_yaml, create_directiories

In [7]:
class ConfigurationManger:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directiories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directiories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_url=  config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        
        return data_ingestion_config


In [8]:
import os
import urllib.request as request
import zipfile
from Red_Wine import logger
from Red_Wine.utils.common import get_size


In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url= self.config.source_url,
                filename= self.config.local_data_file
            )
            logger.info(f'{filename} downloaded with  following info: \n{headers}')
        else:
            logger.info(f'File already exists of size {get_size(Path(self.config.local_data_file))}')


    def extract_zip_file(self):
        '''
        zip file_path: str
        Extract the zip file into the data directory
        Function returns None
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

    


In [10]:
try:
    config = ConfigurationManger()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e



[2024-05-28 09:05:53,259: INFO: common: yaml file:config\config.yaml loaded successfully]
[2024-05-28 09:05:53,265: INFO: common: yaml file:params.yaml loaded successfully]
[2024-05-28 09:05:53,265: INFO: common: yaml file:schema.yaml loaded successfully]
[2024-05-28 09:05:53,273: INFO: common: directory artifacts created]
[2024-05-28 09:05:53,274: INFO: common: directory artifacts/data_ingestion created]
[2024-05-28 09:05:54,522: INFO: 3151101451: artifacts/data_ingestion/data.zip downloaded with  following info: 
Connection: close
Content-Length: 26176
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "f751ec0917e15a3dc07c3094a49fb99713109dd8a9f29150bb5d187ec17facdf"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6DC4:8CAFE:26569A:34C632:66555118
Accept-Ranges: bytes
Date: Tue, 28 May 2024 